In [ ]:
# https://www.sc-best-practices.org/conditions/gsea_pathway.html#id380
# Kang HM, Subramaniam M, Targ S, et al. Multiplexed droplet single-cell RNA-sequencing using natural genetic variation
#   Nat Biotechnol. 2020 Nov;38(11):1356]. Nat Biotechnol. 2018;36(1):89-94. doi:10.1038/nbt.4042

In [ ]:
model_kind = "ivae_kegg"
debug = True

In [ ]:
%load_ext autoreload
%autoreload 2

import scanpy as sc
import tensorflow as tf
from tensorflow.keras.models import Model
from ivae_scorer.datasets import load_kang
from tensorflow.keras import callbacks
import shutil
from ivae_scorer.utils import set_all_seeds
from ivae_scorer.bio import (
    get_adj_matrices,
    sync_gexp_adj,
    build_hipathia_renamers,
    get_reactome_adj,
)
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import minmax_scale
import matplotlib.pyplot as plt
import seaborn as sns
import dotenv
from pathlib import Path
import pandas as pd
import numpy as np
from scipy.stats import weightedtau
import keras.metrics


project_path = Path(dotenv.find_dotenv()).parent
results_path = project_path.joinpath("results")
results_path.mkdir(exist_ok=True, parents=True)
data_path = project_path.joinpath("data")
data_path.mkdir(exist_ok=True, parents=True)
figs_path = results_path.joinpath("figs")
figs_path.mkdir(exist_ok=True, parents=True)
tables_path = results_path.joinpath("tables")
tables_path.mkdir(exist_ok=True, parents=True)

set_all_seeds(seed=42)

tf.config.experimental.enable_op_determinism()

sc.set_figure_params(dpi=300, color_map="inferno")
sc.settings.verbosity = 1
sc.logging.print_header()

In [ ]:
if debug:
    N_EPOCHS = 2
    N_ITERS = 10
else:
    N_EPOCHS = 300
    N_ITERS = 100

if model_kind == "ivae_kegg":
    n_encoding_layers = 3
elif model_kind == "ivae_reactome":
    n_encoding_layers = 2
else:
    raise NotImplementedError(f"{model_kind} not implemented yet.")

print(f"{debug=} {model_kind=}")

In [ ]:
adata = load_kang(data_folder=data_path, normalize=True, n_genes=4000)

In [ ]:
x_trans = adata.to_df()

In [ ]:
circuit_adj, circuit_to_pathway_adj = get_adj_matrices(
    gene_list=x_trans.columns.to_list()
)

circuit_renamer, pathway_renamer, circuit_to_effector = build_hipathia_renamers()

kegg_circuit_names = circuit_adj.rename(columns=circuit_renamer).columns

kegg_pathway_names = circuit_to_pathway_adj.rename(columns=pathway_renamer).columns

circuit_adj.head()

In [ ]:
reactome = get_reactome_adj()
reactome_pathway_names = reactome.columns

In [ ]:
if model_kind == "ivae_kegg":
    x_trans, circuit_adj = sync_gexp_adj(gexp=x_trans, adj=circuit_adj)
elif model_kind == "ivae_reactome":
    x_trans, reactome = sync_gexp_adj(x_trans, reactome)

In [ ]:
from ivae_scorer.models import build_kegg_vae, build_reactome_vae


def get_importances(data, abs=False):
    if abs:
        return np.abs(data).mean(axis=0)
    else:
        return data.mean(axis=0)


def get_activations(act_model, layer_id, data):
    data_encoded = act_model.predict(data)[layer_id]
    return data_encoded

In [ ]:
def train_val_test_split(features, val_size, test_size, stratify, seed):
    train_size = 1 - (val_size + test_size)

    x_train, x_test, y_train, y_test = train_test_split(
        features,
        stratify,
        train_size=train_size,
        stratify=stratify,
        random_state=seed,
    )

    x_val, x_test = train_test_split(
        x_test,
        test_size=test_size / (test_size + val_size),
        stratify=y_test,
        random_state=seed,
    )

    x_train = x_train.astype("float32")
    x_val = x_val.astype("float32")
    x_test = x_test.astype("float32")

    return x_train, x_val, x_test

In [ ]:
results_path_model = results_path.joinpath(model_kind)
obs = adata.obs.copy()

if results_path_model.exists():
    shutil.rmtree(results_path_model)
results_path_model.mkdir(exist_ok=True, parents=True)

In [ ]:
for seed in range(N_ITERS):
    tf.keras.backend.clear_session()

    x_train, x_val, x_test = train_val_test_split(
        x_trans.apply(minmax_scale),
        val_size=0.20,
        test_size=0.20,
        stratify=obs["cell_type"].astype(str) + obs["condition"].astype(str),
        seed=42,
    )

    if model_kind == "ivae_kegg":
        vae, encoder, decoder = build_kegg_vae(
            circuits=circuit_adj, pathways=circuit_to_pathway_adj, seed=seed
        )
    elif model_kind == "ivae_reactome":
        vae, encoder, decoder = build_reactome_vae(reactome, seed=seed)
    else:
        raise NotImplementedError("Model not yet implemented.")

    batch_size = 32

    callback = callbacks.EarlyStopping(
        monitor="val_loss",  # Stop training when `val_loss` is no longer improving
        min_delta=1e-1,  # "no longer improving" being defined as "no better than 1e-5 less"
        patience=100,  # "no longer improving" being further defined as "for at least 3 epochs"
        verbose=0,
    )

    history = vae.fit(
        x_train.values,
        shuffle=True,
        verbose=0,
        epochs=N_EPOCHS,
        batch_size=batch_size,
        callbacks=[callback],
        validation_data=(x_val.values, None),
    )

    evaluation = {}
    evaluation["train"] = vae.evaluate(
        x_train, vae.predict(x_train), verbose=0, return_dict=True
    )
    evaluation["val"] = vae.evaluate(
        x_val, vae.predict(x_val), verbose=0, return_dict=True
    )
    evaluation["test"] = vae.evaluate(
        x_test, vae.predict(x_test), verbose=0, return_dict=True
    )

    pd.DataFrame.from_dict(evaluation).reset_index(names="metric").assign(
        seed=seed
    ).melt(
        id_vars=["seed", "metric"],
        value_vars=["train", "val", "test"],
        var_name="split",
        value_name="score",
    ).assign(
        model=model_kind
    ).to_pickle(
        results_path_model.joinpath(f"metrics-seed-{seed:02d}.pkl")
    )

    layer_outputs = [layer.output for layer in encoder.layers]
    activation_model = Model(inputs=encoder.input, outputs=layer_outputs)

    # only analyze informed and funnel layers
    for layer_id in range(1, len(layer_outputs)):
        if model_kind == "ivae_kegg":
            if layer_id == 1:
                colnames = kegg_circuit_names
                layer_name = "circuits"
            elif layer_id == 2:
                colnames = kegg_pathway_names
                layer_name = "pathways"
            elif layer_id == (len(layer_outputs) - 1):
                n_latents = len(kegg_pathway_names) // 2
                colnames = [f"latent_{i:02d}" for i in range(n_latents)]
                layer_name = "funnel"
            else:
                continue
        elif model_kind == "ivae_reactome":
            if layer_id == 1:
                colnames = reactome_pathway_names
                layer_name = "pathways"
            elif layer_id == (len(layer_outputs) - 1):
                n_latents = len(reactome_pathway_names) // 2
                colnames = [f"latent_{i:02d}" for i in range(n_latents)]
                layer_name == "funnel"
            else:
                continue
        else:
            raise NotImplementedError("Model not yet implemented.")

        print(f"encoding layer {layer_id}")

        encodings = get_activations(
            act_model=activation_model,
            layer_id=layer_id,
            data=x_trans.apply(minmax_scale),
        )
        encodings = pd.DataFrame(encodings, index=x_trans.index, columns=colnames)
        encodings["split"] = "train"
        encodings.loc[x_val.index, "split"] = "val"
        encodings.loc[x_test.index, "split"] = "test"
        encodings["layer"] = layer_name
        encodings["seed"] = seed
        encodings["model"] = model_kind
        encodings = encodings.merge(
            obs[["cell_type", "condition"]],
            how="left",
            left_index=True,
            right_index=True,
        )
        encodings.to_pickle(
            results_path_model.joinpath(
                f"encodings_layer-{layer_id:02d}_seed-{seed:02d}.pkl"
            )
        )

In [ ]:
dot_img_file = results_path_model.joinpath(f"architecture.{model_kind}.png")
keras.utils.plot_model(vae, to_file=dot_img_file, show_shapes=True)

In [ ]:
non_layer_names = ["split", "layer", "seed", "cell_type", "condition", "model"]

In [ ]:
scores_metrics = [
    pd.read_pickle(results_path_model.joinpath(f"metrics-seed-{seed:02d}.pkl"))
    for seed in range(N_ITERS)
]
scores_metrics = pd.concat(scores_metrics, axis=0, ignore_index=True)
scores_metrics.to_pickle(results_path_model.joinpath("scores_metrics.pkl"))

scores_metrics.head()

In [ ]:
custom_params = {"axes.spines.right": False, "axes.spines.top": False}
sns.set_theme(context="paper", font_scale=0.5, style="ticks", rc=custom_params)

g = sns.catplot(
    data=scores_metrics,
    kind="violin",
    col="metric",
    height=2,
    aspect=0.9,
    sharey=False,
    x="model",
    y="score",
    hue="split",
    split=False,
    cut=0,
    fill=False,
    density_norm="count",
    inner="quart",
    linewidth=0.5,
)

In [ ]:
scores_informed = {}

for layer_id in range(1, n_encoding_layers + 1):
    if results_path_model.joinpath(
        f"encodings_layer-{layer_id:02d}_seed-00.pkl"
    ).exists():
        results_layer = [
            pd.read_pickle(
                results_path_model.joinpath(
                    f"encodings_layer-{layer_id:02d}_seed-{seed:02d}.pkl"
                )
            )
            for seed in range(N_ITERS)
        ]
    else:
        continue

    scores_informed[layer_id] = {}
    for split in ["train", "test", "val"]:
        results = [
            x.loc[x["split"] == split].drop(non_layer_names, axis=1)
            for x in results_layer
        ]
        scores_informed[layer_id][split] = []
        for seed_i in range(N_ITERS):
            for seed_j in range(seed_i + 1, N_ITERS):
                scores_informed[layer_id][split].append(
                    weightedtau(
                        get_importances(data=results[seed_i], abs=True),
                        get_importances(data=results[seed_j], abs=True),
                    )[0]
                )

In [ ]:
scores_informed = (
    pd.DataFrame.from_dict(scores_informed)
    .melt(var_name="layer", value_name="score", ignore_index=False)
    .reset_index(names=["split"])
    .explode("score")
)
scores_informed["score"] = scores_informed["score"].astype("float")
scores_informed["model"] = model_kind
scores_informed.to_pickle(results_path_model.joinpath("scores_informed.pkl"))

In [ ]:
scores_informed.head()

In [ ]:
custom_params = {"axes.spines.right": False, "axes.spines.top": False}
sns.set_theme(context="paper", font_scale=0.5, style="ticks", rc=custom_params)
plt.figure(figsize=(2, 2))
sns.violinplot(
    data=scores_informed,
    x="layer",
    y="score",
    hue="split",
    split=False,
    cut=0,
    fill=False,
    density_norm="count",
    inner="quart",
    linewidth=0.5,
)
sns.despine()

In [ ]:
from sklearn.cluster import MiniBatchKMeans
from multiprocessing import cpu_count
from sklearn.metrics.cluster import adjusted_mutual_info_score

batch_size = 256 * cpu_count() + 1

In [ ]:
clust_scores = {}

for layer_id in range(1, n_encoding_layers + 1):
    if results_path_model.joinpath(
        f"encodings_layer-{layer_id:02d}_seed-00.pkl"
    ).exists():
        results_layer = [
            pd.read_pickle(
                results_path_model.joinpath(
                    f"encodings_layer-{layer_id:02d}_seed-{seed:02d}.pkl"
                )
            )
            for seed in range(N_ITERS)
        ]
    else:
        continue

    train_embeddings_lst = [
        x.loc[(x["split"] == "train") & (x["condition"] == "control")]
        for x in results_layer
    ]
    val_embeddings_lst = [
        x.loc[(x["split"] == "val") & (x["condition"] == "control")]
        for x in results_layer
    ]
    test_embeddings_lst = [
        x.loc[(x["split"] == "test") & (x["condition"] == "control")]
        for x in results_layer
    ]

    clust_scores[layer_id] = {}
    clust_scores[layer_id]["train"] = []
    clust_scores[layer_id]["val"] = []
    clust_scores[layer_id]["test"] = []

    for seed in range(N_ITERS):
        y_train = train_embeddings_lst[seed]["cell_type"]
        y_val = val_embeddings_lst[seed]["cell_type"]
        y_test = test_embeddings_lst[seed]["cell_type"]

        train_embeddings = train_embeddings_lst[seed].drop(non_layer_names, axis=1)
        val_embeddings = val_embeddings_lst[seed].drop(non_layer_names, axis=1)
        test_embeddings = test_embeddings_lst[seed].drop(non_layer_names, axis=1)

        model = MiniBatchKMeans(n_clusters=y_train.nunique(), batch_size=batch_size)
        model.fit(train_embeddings)
        clust_scores[layer_id]["train"].append(
            adjusted_mutual_info_score(y_train, model.labels_)
        )
        clust_scores[layer_id]["val"].append(
            adjusted_mutual_info_score(y_val, model.predict(val_embeddings))
        )
        clust_scores[layer_id]["test"].append(
            adjusted_mutual_info_score(y_test, model.predict(test_embeddings))
        )

In [ ]:
clust_scores = (
    pd.DataFrame.from_dict(clust_scores)
    .melt(var_name="layer", value_name="score", ignore_index=False)
    .reset_index(names=["split"])
    .explode("score")
)
clust_scores["score"] = clust_scores["score"].astype("float")
clust_scores["model"] = model_kind
clust_scores.to_pickle(results_path_model.joinpath("scores_clustering.pkl"))

In [ ]:
clust_scores.head()

In [ ]:
custom_params = {"axes.spines.right": False, "axes.spines.top": False}
sns.set_theme(context="paper", font_scale=0.5, style="ticks", rc=custom_params)
plt.figure(figsize=(2, 2))
sns.violinplot(
    data=clust_scores,
    x="layer",
    y="score",
    hue="split",
    split=False,
    cut=0,
    fill=False,
    density_norm="count",
    inner="quart",
    linewidth=0.5,
)
sns.despine()